In [1]:
%load_ext line_profiler
from adapter import mkcalc
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import sys
import time

In [2]:
def analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500):
    adap = mkcalc.AsympMK(B=bgs,gam_neg=gam,theta_f=0.001,alLow=alweak,alTot=al,neut_mid=False,L_mid=lCoding,Lf=lNonCoding,N=popsize,nsim=2500,pref="unc",gL=10,n=sample,gH=500)
    
    adap.set_theta_f()
    theta_f = adap.theta_f
    adap.B = 0.999
    adap.set_theta_f()
    adap.setPpos()
    adap.theta_f = theta_f
    adap.B = bgs

    pos = adap.alx(adap.gL,adap.gH,adap.pposL,adap.pposH)
    return(theta_f)

In [3]:
%timeit analyticalTime()

2.11 s ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Solving original approximation. Using default parameters provided at *al_x.py*: no bgs and 20% of alpha due to weak selection

In [11]:
%lprun -f analyticalTime analyticalTime()

Timer unit: 1e-06 s

Total time: 6.24656 s
File: <ipython-input-9-b64ab378000c>
Function: analyticalTime at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500):
     2         1         64.0     64.0      0.0      adap = mkcalc.AsympMK(B=bgs,gam_neg=gam,theta_f=0.001,alLow=alweak,alTot=al,neut_mid=False,L_mid=lCoding,Lf=lNonCoding,N=popsize,nsim=2500,pref="unc",gL=10,n=sample,gH=500)
     3                                               
     4         1        224.0    224.0      0.0      adap.set_theta_f()
     5         1          1.0      1.0      0.0      theta_f = adap.theta_f
     6         1          1.0      1.0      0.0      adap.B = 0.999
     7         1        138.0    138.0      0.0      adap.set_theta_f()
     8         1     514715.0 514715.0      8.2      adap.setPpos()
  

**In order to get the alpha(x) and the expected values of each category (Dn,Ds,Pn,Ps) we got a mean of 3.47s per iteration when using Ne=500 n=250**. I changed the original values to show which are the variables influencing the computing time. As the expectation is calculated for each category of the frequency spectrum, population and sample size must be the limiting variables in the process. Scale or shape parameters of gamma's distrubutions should not influence in computation time since equation will be solve for any parameter equally.

#### Changing gamma values

In [5]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-200,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500)

3.44 s ± 50.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=20,gamStrong=500)

3.41 s ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=400)

3.57 s ± 49.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-200,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=40,gamStrong=600)

3.53 s ± 74.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing Ne in a half

In [25]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=250,sample=250,lCoding=501,lNonCoding=10**6)

1.95 s ± 35.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing n in a half

In [16]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=175,lCoding=501,lNonCoding=10**6)

3.21 s ± 92.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Duplicating Ne and n values

In [17]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=1000,sample=500,lCoding=501,lNonCoding=10**6)

8.21 s ± 52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing length in a half

In [22]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=250,lNonCoding=5*10**5)

3.34 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Duplicating length values

In [24]:
%%timeit
analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=1001,lNonCoding=2*10**6)

3.35 s ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Checking the process individually

In [10]:
%lprun -f analyticalTime analyticalTime()

Timer unit: 1e-06 s

Total time: 11.2382 s
File: <ipython-input-2-25f7f86dd27a>
Function: analyticalTime at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500):
     2         1          2.0      2.0      0.0      B  = float(0.999)
     3         1          1.0      1.0      0.0      al = float(0.2)
     4         1          1.0      1.0      0.0      weak=0.2
     5         1         58.0     58.0      0.0      adap = mkcalc.AsympMK(B=B,gam_neg=gam,theta_f=0.001,alLow=alweak,alTot=al,neut_mid=False,L_mid=lCoding,Lf=lNonCoding,N=popsize,nsim=2500,pref="unc",gL=10,n=sample,gH=500)
     6         1        217.0    217.0      0.0      adap.set_theta_f()
     7         1          1.0      1.0      0.0      theta_f = adap.theta_f
     8         1          1.0      1.0      0.0      theta_f = a

setPpos and alpha calculations are the most problematic functions. Both need almost a second to run with Ne=500 and n=250. I compiled the functions manually with originals values to run step-by-step in order to check where are the bottlenecks

### Exploring computation times on *set_theta_f* and *alx* functions

In [1]:
%load_ext line_profiler
from adapter import mkcalc
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import sys
import time
import inspect

In [2]:
import sys
sys.path.insert(0, '/home/jmurga/mktest/scripts/')
from mkcalcDef import *

/home/jmurga/anaconda3/envs/mktest/lib/python3.8/site-packages/julia/core.py:685: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


In [3]:
adap = mkcalc.AsympMK(B=0.999,gam_neg=-83,theta_f=0.001,alLow=0.2,alTot=0.2,neut_mid=False,L_mid=501,Lf=10**6,N=500,nsim=2500,pref="unc",gL=10,n=250,gH=500)

In [3]:
%lprun -f set_theta_f_manual set_theta_f_manual()

Timer unit: 1e-06 s

Total time: 0.000262 s
File: /home/jmurga/mktest_dev/scripts/mkcalcDef.py
Function: set_theta_f_manual at line 52

Line #      Hits         Time  Per Hit   % Time  Line Contents
    52                                           def set_theta_f_manual():
    53         1        259.0    259.0     98.9  	theta_f  = fsolve(lambda theta: Br_manual(Lf,theta)-B,0.00001)
    54         1          3.0      3.0      1.1  	theta_f = theta_f[0]

**I think *set_theta_f* could not be optimize. It's just solving an equation with *x* parameters, in that sense only depends on *fsolve* computation time.**

In [8]:
print(inspect.getsource(set_theta_f_manual))

def set_theta_f():
	theta_f  = fsolve(lambda theta: Br(Lf,theta)-B,0.00001)
	theta_f = theta_f[0]



In [9]:
print(inspect.getsource(Br_manual))

def Br(Lmax,theta):
	t = -1.*gam_neg/(NN+0.)
	u = theta/(2.*NN)
	r = rho/(2.*NN)
	return np.exp(-4*u*Lmax/(2*Lmax*r+t))



In [6]:
print(inspect.getsource(alx_manual))

def alx_manual(gammaL,gammaH,pposL,pposH):
	ret = []

	#Fixation
	fN = B*fixNeut_manual()
	fNeg = B*fixNegB_manual(0.5*pposH+0.5*pposL)
	fPosL = fixPosSim_manual(gammaL,0.5*pposL)
	fPosH = fixPosSim_manual(gammaH,0.5*pposH)

	#Pol
	neut = cumuSfs(DiscSFSNeutDown_manual())
	selH = cumuSfs(DiscSFSSelPosDown_manual(gammaH,pposH))
	selL = cumuSfs(DiscSFSSelPosDown_manual(gammaL,pposL))
	selN = cumuSfs(DiscSFSSelNegDown_manual(pposH+pposL))
	
	sel = []
	for i in range(0,len(selH)):
		sel.append((selH[i]+selL[i])+selN[i])
	for i in range(0,nn-1):
		ret.append(float(1. - (fN/(fPosL + fPosH+  fNeg+0.))* sel[i]/neut[i]))
	return (ret,sel,neut,selH,selL,selN,fN,fNeg,fPosL,fPosH)



In [3]:
%lprun -f alx_manual alx_manual(gL,gH,pposL,pposH)

Timer unit: 1e-06 s

Total time: 5.92915 s
File: /home/jmurga/mktest_dev/scripts/mkcalcDef.py
Function: alx_manual at line 163

Line #      Hits         Time  Per Hit   % Time  Line Contents
   163                                           def alx_manual(gammaL,gammaH,pposL,pposH):
   164         1          1.0      1.0      0.0  	ret = []
   165                                           
   166                                           	#Fixation
   167         1          3.0      3.0      0.0  	fN = B*fixNeut_manual()
   168         1       7290.0   7290.0      0.1  	fNeg = B*fixNegB_manual(0.5*pposH+0.5*pposL)
   169         1        417.0    417.0      0.0  	fPosL = fixPosSim_manual(gammaL,0.5*pposL)
   170         1      10740.0  10740.0      0.2  	fPosH = fixPosSim_manual(gammaH,0.5*pposH)
   171                                           
   172                                           	#Pol
   173         1     275816.0 275816.0      4.7  	neut = cumuSfs_manual(DiscSFSNeutDown_

In [4]:
%timeit fN2 = B*fixNeut_manual()
%timeit fNeg2 = B*fixNegB_manual(0.5*pposH+0.5*pposL)
%timeit fPosL2 = fixPosSim_manual(gL,0.5*pposL)
%timeit fPosH2 = fixPosSim_manual(gH,0.5*pposH)
%timeit neut2 = cumuSfs_manual(DiscSFSNeutDown_manual())
%timeit selH2 = cumuSfs_manual(DiscSFSSelPosDown_manual(gH,pposH))
%timeit selL2 = cumuSfs_manual(DiscSFSSelPosDown_manual(gL,pposL))
%timeit selN2 = cumuSfs_manual(DiscSFSSelNegDown_manual(pposH+pposL))

140 ns ± 2.96 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
973 µs ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
79.9 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.65 ms ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
244 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
279 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
279 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.21 s ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
print(inspect.getsource(DiscSFSSelNegDown_manual))

def DiscSFSSelNegDown_manual(ppos):
		return B*(theta_mid_neutral)*0.745*(np.dot(binomOp(),DiscSFSSelNeg(ppos)))[1:-1]



In [13]:
print(inspect.getsource(DiscSFSSelNeg_manual))

def DiscSFSSelNeg_manual(ppos):
	NN2 = int(round(NN*B))
	dFunc = np.vectorize(FullNeg)
	return np.multiply(1./(NN2+0.),dFunc(ppos,[i/(NN2+0.) for i in range(0,NN2+1)]))



In [18]:
print(inspect.getsource(FullNeg_manual))

def FullNeg_manual( ppos, x):
	beta = be/(1.*B)
	if x > 0 and x < 1.:
		return (1.-ppos)*(2.**-al)*(beta**al)*(-mpmath.zeta(al,x+beta/2.) + mpmath.zeta(al,(2+beta)/2.))/((-1.+x)*x)

	return 0.



Although it is vectorice each computation solving gamma on negative distribution is quite slow

In [3]:
%timeit DiscSFSSelNegDown_manual(pposH+pposL)

1.17 s ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Julia implemenation

In [6]:
%timeit DiscSFSSelNegDown_manual2(pposH+pposL)

240 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
np.array([i/(NN2+0.) for i in range(0,NN2+1)])[:10]

array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901])

In [5]:
NN2 = int(round(NN*B))
dFuncVector = np.vectorize(FullNeg_manual)

In [6]:
%timeit y = dFuncVector(pposH+pposL,[i/(NN2+0.) for i in range(0,NN2+1)])

1.02 s ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Checking one iteration time

In [20]:
x = [i/(NN2+0.) for i in range(0,NN2+1)]

In [22]:
%timeit  b = FullNeg_manual(pposH+pposL,x[1])

1.23 ms ± 44.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Numba compilation with allocated variables
Although python compilers (cython, numba...) could be a good solutions to improve performance, most of them will not be easy to apply on the code. In addition they only usually accept pure python code, so most parts on the code cannot be compiled. I perform a little approach with Numba which can deal with numpy arrays. I allocated variables and changed list comprehension to boost performance. But we have the same problem, solving the equations with scipy/mpmath is quite slow.

In [4]:
fixNeut_numba()
%timeit B*fixNeut_manual()
%timeit B*fixNeut_numba()

131 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
80.5 ns ± 0.278 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [5]:
fNeg          = B*fixNegB_numba(0.5*pposH+0.5*pposL)
%timeit fNeg  = B*fixNegB_manual(0.5*pposH+0.5*pposL)
%timeit fNeg  = B*fixNegB_numba(0.5*pposH+0.5*pposL)

879 µs ± 6.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
884 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
fPosH= fixPosSim_numba(gL,pposL)
%timeit fPosH = fixPosSim_manual(gL,0.5*pposL)
%timeit fPosH = fixPosSim_numba(gL,0.5*pposL)

73.6 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
31.9 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
